In [ ]:
!wget -O data.zip "https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/4drtyfjtfy-1.zip"

--2022-01-09 12:38:54--  https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/4drtyfjtfy-1.zip
Resolving md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)... 52.218.121.234
Connecting to md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)|52.218.121.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95592747 (91M) [application/octet-stream]
Saving to: ‘data.zip’

data.zip            100%[===================>]  91.16M  21.8MB/s    in 4.6s    

2022-01-09 12:38:59 (19.9 MB/s) - ‘data.zip’ saved [95592747/95592747]



In [ ]:
!unzip data.zip

Archive:  data.zip
  inflating: dataset2.zip            


In [ ]:
!unzip dataset2.zip

Archive:  dataset2.zip
   creating: dataset2/
  inflating: dataset2/cloudy1.jpg    
  inflating: dataset2/cloudy10.jpg   
  inflating: dataset2/cloudy100.jpg  
  inflating: dataset2/cloudy101.jpg  
  inflating: dataset2/cloudy102.jpg  
  inflating: dataset2/cloudy103.jpg  
  inflating: dataset2/cloudy104.jpg  
  inflating: dataset2/cloudy105.jpg  
  inflating: dataset2/cloudy106.jpg  
  inflating: dataset2/cloudy107.jpg  
  inflating: dataset2/cloudy108.jpg  
  inflating: dataset2/cloudy109.jpg  
  inflating: dataset2/cloudy11.jpg   
  inflating: dataset2/cloudy110.jpg  
  inflating: dataset2/cloudy111.jpg  
  inflating: dataset2/cloudy112.jpg  
  inflating: dataset2/cloudy113.jpg  
  inflating: dataset2/cloudy114.jpg  
  inflating: dataset2/cloudy115.jpg  
  inflating: dataset2/cloudy116.jpg  
  inflating: dataset2/cloudy117.jpg  
  inflating: dataset2/cloudy118.jpg  
  inflating: dataset2/cloudy119.jpg  
  inflating: dataset2/cloudy12.jpg   
  inflating: dataset2/cloudy120.jpg  
  in

In [ ]:
!wget -O cup.jpeg "https://lh5.googleusercontent.com/sTDIn5BK6czLJsB9u-m-VcLma4a6IbJ6rKL0NC3Vp8XESJuAoc5detOT9wtNobOX2CvCX4Qtel21vr7R3Jnv13MrbNIy-yhb-4z6H08wIthMXhce8mjglvufpLOzgx0ftFbvGphn"

--2022-01-09 12:39:13--  https://lh5.googleusercontent.com/sTDIn5BK6czLJsB9u-m-VcLma4a6IbJ6rKL0NC3Vp8XESJuAoc5detOT9wtNobOX2CvCX4Qtel21vr7R3Jnv13MrbNIy-yhb-4z6H08wIthMXhce8mjglvufpLOzgx0ftFbvGphn
Resolving lh5.googleusercontent.com (lh5.googleusercontent.com)... 64.233.181.132, 2607:f8b0:4001:c09::84
Connecting to lh5.googleusercontent.com (lh5.googleusercontent.com)|64.233.181.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38944 (38K) [image/jpeg]
Saving to: ‘cup.jpeg’

cup.jpeg            100%[===================>]  38.03K  --.-KB/s    in 0s      

2022-01-09 12:39:13 (101 MB/s) - ‘cup.jpeg’ saved [38944/38944]



In [ ]:
import glob, os, shutil

In [ ]:
types = [ ("Cloudy","cloud*.*"), ("Rainy","rain*.*"), ("Shiny", "shin*.*"), ("Sunrise","sunrise*.*")  ]

shutil.rmtree("dataset", ignore_errors=True)
for folder_name, file_mask in types:
  files = sorted(glob.glob("dataset2/"+file_mask))
  os.makedirs(f"dataset/Train/{folder_name}")
  os.makedirs(f"dataset/Test/{folder_name}")
  train_qnty = int(0.8*len(files))
  [shutil.copy(item, f"dataset/Train/{folder_name}") for item in files[:train_qnty] ]
  [shutil.copy(item, f"dataset/Test/{folder_name}") for item in files[train_qnty:] ]
  print(folder_name, file_mask, len(files))

Cloudy cloud*.* 300
Rainy rain*.* 215
Shiny shin*.* 253
Sunrise sunrise*.* 357


In [ ]:

from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras import Input
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
my_image = load_img('cup.jpeg', target_size=(224, 224))
my_image = img_to_array(my_image)
my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
my_image = preprocess_input(my_image)

In [ ]:
#instantiate the model
model = VGG16()
#make predictions
prediction = model.predict(my_image)

In [ ]:

prediction = decode_predictions(prediction)

item = prediction[0][0]
 
# print the result
print(f"{item[1]} with a probability of {int(item[2]*100)}%")

coffee_mug with a probability of 25%


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras import Input
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.layers import Dense, Flatten

In [ ]:
model = VGG16(include_top=False, input_shape=(244, 244, 3))

for layer in model.layers:
	layer.trainable = False

hidden = Flatten()(model.output)
outputs = Dense(4, activation='softmax')(hidden)

model = Model(inputs=model.input, outputs=outputs)

model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 122, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 122, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 122, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 61, 61, 128)       0   

In [ ]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [ ]:

train_data_gen = ImageDataGenerator(rescale=1.0/255, 
                               shear_range=0.5,
                                zoom_range=0.7, 
                                horizontal_flip=True,
                                vertical_flip=True)

test_data_gen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_data = train_data_gen.flow_from_directory('dataset/Train',
                                               target_size=(244, 244),
                                               class_mode='categorical')
 
test_data = test_data_gen.flow_from_directory('dataset/Test',
                                               target_size=(244, 244),
                                               class_mode='categorical')

Found 899 images belonging to 4 classes.
Found 226 images belonging to 4 classes.


In [ ]:

history = model.fit_generator(train_data, validation_data=test_data, epochs=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
29/29 [==============================] - 40s 925ms/step - loss: 1.1457 - accuracy: 0.5462 - val_loss: 1.1041 - val_accuracy: 0.5000
Epoch 2/5
29/29 [==============================] - 24s 818ms/step - loss: 0.6351 - accuracy: 0.7564 - val_loss: 0.4385 - val_accuracy: 0.8496
Epoch 3/5
29/29 [==============================] - 24s 837ms/step - loss: 0.4689 - accuracy: 0.8231 - val_loss: 0.4737 - val_accuracy: 0.8363
Epoch 4/5
29/29 [==============================] - 24s 819ms/step - loss: 0.3786 - accuracy: 0.8643 - val_loss: 0.3898 - val_accuracy: 0.8496
Epoch 5/5
29/29 [==============================] - 24s 826ms/step - loss: 0.3877 - accuracy: 0.8509 - val_loss: 0.3413 - val_accuracy: 0.8761


In [ ]:
!wget -O "final-test.jpeg" "https://lh4.googleusercontent.com/u4XKf1p2kwK3Of6a-2a_GdPzTpNSRAmRDP_hNckOUsYW0Y7PMQiUxJ1m5bsVxrCqMTD8VFK65XiBfJqGxbjTOgQWxIQY61s_xPR98Lb4EdpJhxbzP1AM2_wlZGcWWBr__hIb3rLr"

--2022-01-09 12:41:49--  https://lh4.googleusercontent.com/u4XKf1p2kwK3Of6a-2a_GdPzTpNSRAmRDP_hNckOUsYW0Y7PMQiUxJ1m5bsVxrCqMTD8VFK65XiBfJqGxbjTOgQWxIQY61s_xPR98Lb4EdpJhxbzP1AM2_wlZGcWWBr__hIb3rLr
Resolving lh4.googleusercontent.com (lh4.googleusercontent.com)... 64.233.181.132, 2607:f8b0:4001:c09::84
Connecting to lh4.googleusercontent.com (lh4.googleusercontent.com)|64.233.181.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67482 (66K) [image/jpeg]
Saving to: ‘final-test.jpeg’

final-test.jpeg     100%[===================>]  65.90K  --.-KB/s    in 0.001s  

2022-01-09 12:41:49 (97.8 MB/s) - ‘final-test.jpeg’ saved [67482/67482]



In [ ]:

my_image = load_img('final-test.jpeg', target_size=(244, 244))
 

my_image = img_to_array(my_image)
my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
my_image = preprocess_input(my_image)
 

prediction = model.predict(my_image)
prediction

array([[1.9770138e-22, 1.2577141e-11, 3.0737298e-20, 1.0000000e+00]],
      dtype=float32)

In [ ]:
[np.round(x) for x in prediction]

[array([0., 0., 0., 1.], dtype=float32)]

In [ ]:
np.argmax(prediction)

3

In [ ]:
test_data.class_indices

{'Cloudy': 0, 'Rainy': 1, 'Shiny': 2, 'Sunrise': 3}

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 122, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 122, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 122, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 61, 61, 128)       0   

In [ ]:
model.save("clouds.h5")